In [2]:
import numpy as np
import pandas as pd
from statsmodels.tsa.stattools import pacf

In [4]:
def _prepare_data_corr_plot(x, lags, zero):
    x = pd.Series(x)
    zero = bool(zero)
    irregular = False if zero else True
    if lags is None:
        # GH 4663 - use a sensible default value
        nobs = x.shape[0]
        lim = min(int(np.ceil(10 * np.log10(nobs))), nobs - 1)
        lags = np.arange(not zero, lim + 1)
    elif np.isscalar(lags):
        lags = np.arange(not zero, int(lags) + 1)  # +1 for zero lag
    else:
        irregular = True
        lags = np.asanyarray(lags).astype(int)
    nlags = lags.max(0)

    return lags, nlags, irregular

In [3]:
def window(
    x,
    ax=None,
    lags=None,
    alpha=0.05,
    method="ywm",
    use_vlines=True,
    title="Partial Autocorrelation",
    zero=True,
    vlines_kwargs=None,
    **kwargs,
):


    lags, nlags, irregular = _prepare_data_corr_plot(x, lags, zero)

    confint = None
    if alpha is None:
        acf_x = pacf(x, nlags=nlags, alpha=alpha, method=method)
    else:
        acf_x, confint = pacf(x, nlags=nlags, alpha=alpha, method=method)


    confint[:, 0] = confint[:, 0] - acf_x
    confint[:, 1] = confint[:, 1] - acf_x
    confint = pd.DataFrame(confint)
    confint = confint.abs()

    idx = []

    for i in range(0,30):
      if acf_x[i] > max(confint.iloc[i,:]) :
        idx.append(i)

    return idx[-1]